In [24]:
# uncomment the code below to temporarily install active_learning_ratio_estimation from source
# if you do not do this, unless you already have it installed in this environment, this notebook will not run

# import os, sys, tempfile, shutil
# original_dir = os.getcwd()
# REPO_NAME = 'active_learning_ratio_estimation'
# BRANCH = 'master'
# tempdir = tempfile.gettempdir()
# os.chdir(tempdir)
# if os.path.exists(REPO_NAME):
#   print('Removing repo')
#   shutil.rmtree(REPO_NAME)
# print('Cloning repo')
# os.system(f'git clone --single-branch --branch {BRANCH} https://github.com/cjs220/{REPO_NAME}.git')
# sys.path.insert(0, os.path.join(tempdir, REPO_NAME))
# os.chdir(original_dir)

['../..', '..', '..', '..', 'C:\\Users\\h12913cs\\dev\\active_learning_ratio_estimation\\examples', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\python37.zip', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\DLLs', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\lib', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37', '', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\lib\\site-packages', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\lib\\site-packages\\win32', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\lib\\site-packages\\win32\\lib', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\lib\\site-packages\\Pythonwin', 'C:\\Users\\h12913cs\\AppData\\Local\\Continuum\\miniconda3\\envs\\py37\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\h12913cs\\.ipython']


In [4]:
# matplotlib setup
import matplotlib.pyplot as plt

%matplotlib inline

size=24
params = {
    'legend.fontsize': size*0.75,
    'figure.figsize': (10, 5),
    'axes.labelsize': size,
    'axes.titlesize': size,
    'xtick.labelsize': size * 0.75,
    'ytick.labelsize': size * 0.75,
    'font.family': 'sans-serif',
    'axes.titlepad': 12.5
}
plt.rcParams.update(params)

In [5]:
# logging setup
import logging

# make logger output only INFO from active_learning_ratio_estimation
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

for key in logging.Logger.manager.loggerDict:
    if "active_learning_ratio_estimation" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


def triple_mixture(gamma):
    mixture_probs = [
        0.5 * (1 - gamma),
        0.5 * (1 - gamma),
        gamma
    ]
    gaussians = [
        tfd.Normal(loc=-2, scale=0.75),
        tfd.Normal(loc=0, scale=2),
        tfd.Normal(loc=1, scale=0.5)
    ]
    dist = tfd.Mixture(
        cat=tfd.Categorical(probs=mixture_probs),
        components=gaussians
    )
    return dist

In [22]:
from active_learning_ratio_estimation.dataset import ParamGrid, ParamIterator, SinglyParameterizedRatioDataset

theta_bounds = (0, 1)
theta_0 = 0.05
num_grid = 11
n_samples_per_theta = int(1e3)
param_grid = ParamGrid(bounds=[theta_bounds], num=num_grid)  # all possible parameter points

theta_1_iterator = ParamGrid(bounds=[theta_bounds], num=3)  # initial parameter points in dataset

# test dataset
test_param_points = 10
test_iterator = ParamIterator([np.random.rand(1) for _ in range(test_param_points)])
test_dataset = SinglyParameterizedRatioDataset.from_simulator(
    simulator_func=triple_mixture,
    theta_0=theta_0,
    theta_1_iterator=test_iterator,
    n_samples_per_theta=n_samples_per_theta,
    include_log_probs=True
)

ModuleNotFoundError: No module named 'active_learning_ratio_estimation.dataset'

In [ ]:
from active_learning_ratio_estimation.model import FlipoutClassifier, SinglyParameterizedRatioModel

estimator = FlipoutClassifier(
    n_hidden=(15, 15),
    epochs=1,
    patience=0,
    validation_split=0.1,
    verbose=0,
)
ratio_model = SinglyParameterizedRatioModel(estimator=estimator)

In [ ]:
from active_learning_ratio_estimation.active_learning import active_learner

learners = dict()
test_mses = dict()
acquisition_functions = [
    'std_regressor',
    'random',
]
n_iter = 5

for acquisition_function in acquisition_functions:
    print(f'\n**** Fitting with {acquisition_function} ****\n')
    active_learner = ActiveLearner(
        simulator_func=triple_mixture,
        theta_0=theta_0,
        theta_1_iterator=theta_1_iterator,
        n_samples_per_theta=n_samples_per_theta,
        ratio_model=ratio_model,
        total_param_grid=param_grid,
        test_dataset=test_dataset,
        acquisition_function=acquisition_function,
        ucb_kappa=0.0,
        validation_mode=True
    )
    active_learner.fit(n_iter)
    learners[acquisition_function] = active_learner
    test_mses[acquisition_function] = active_learner.test_history['mse'].values

In [ ]:
plt.figure()

for af, mse in test_mses.items():
    plt.plot(mse, label=af)

plt.legend()